In [8]:
### INNER ICL

import os
import csv
import random

def load_in_context_examples(example_folder, n=3):
    examples = []

    # Get lists of .txt and .py files
    txt_files = [f for f in os.listdir(example_folder) if f.startswith("prompt_") and f.endswith('.txt')]
    py_files = [f for f in os.listdir(example_folder) if f.startswith("modified_") and f.endswith('.py')]

    # Create a dictionary mapping base filenames to .py files
    py_file_dict = {f.replace("modified_", "").replace(".py", ""): f for f in py_files}

    for txt_file in txt_files:
        base_name = txt_file.replace("prompt_", "").replace(".txt", "")

        if base_name in py_file_dict:
            txt_file_path = os.path.join(example_folder, txt_file)
            py_file_path = os.path.join(example_folder, py_file_dict[base_name])

            # Load the prompt + code from the .txt file
            with open(txt_file_path, 'r', encoding='utf-8') as f:
                prompt_code = f.read().strip()

            # Load the expected code from the .py file
            with open(py_file_path, 'r', encoding='utf-8') as f:
                expected_code = f.read().strip()

            examples.append({
                'prompt_code': prompt_code,
                'expected_code': expected_code
            })

    # Randomly sample n examples
    return random.sample(examples, min(n, len(examples)))

def create_evaluation_dataset(example_folder, target_folder, actual_output_folder, output_csv, n=3):
    dataset = []

    # Get all target prompt .txt files in the target folder
    target_txt_files = [f for f in os.listdir(target_folder) if f.startswith("prompt_") and f.endswith('.txt')]

    for target_txt_file in target_txt_files:
        target_base_name = target_txt_file.replace("prompt_", "").replace(".txt", "")
        target_txt_path = os.path.join(target_folder, target_txt_file)

        # Find the corresponding expected output .py file
        expected_py_file = f"modified_{target_base_name}.py"
        expected_py_path = os.path.join(target_folder, expected_py_file)

        # Find the corresponding actual output .py file
        actual_output_file = f"prompt_{target_base_name}_generated_ICL.py"
        actual_output_path = os.path.join(actual_output_folder, actual_output_file)

        if not os.path.exists(expected_py_path):
            print(f"Expected output file not found for {target_txt_file}")
            continue

        if not os.path.exists(actual_output_path):
            print(f"Actual output file not found for {target_txt_file}")
            continue

        # Load the target prompt code
        with open(target_txt_path, 'r', encoding='utf-8') as f:
            target_prompt_code = f.read().strip()

        # Load the expected output code
        with open(expected_py_path, 'r', encoding='utf-8') as f:
            expected_output_code = f.read().strip()

        # Load the actual output code
        with open(actual_output_path, 'r', encoding='utf-8') as f:
            actual_output_code = f.read().strip()

        # Load in-context examples
        in_context_examples = load_in_context_examples(example_folder, n=n)

        # Construct the input by concatenating in-context examples and the target prompt
        input_text = """
        You are an AI assistant that generates code for inner analyzers using the Mobileinsight-core Python library, a library that enables below-IP, \
        fine-grained mobile network analytics on end devices. It is a cross-platform package for mobile network monitoring and analysis.

        For context, I will be giving a few examples of a prompt + outer analyzer code pairs along with their corresponding expected inner analyzer code.

        Then I will give the main target prompt that you need to follow in order to generate an inner analyzer.

        NOTE: PLEASE PROVIDE ONLY THE CODE AND NOTHING ELSE, AS THIS OUTPUT IS BEING DIRECTLY SAVED TO A .PY FILE AND RAN AUTONOMOUSLY. \
        ADDITIONALLY, ENSURE THAT YOU PROVIDE THE FULL COMPLETE CODE, AND DO NOT LEAVE OUT ANY PARTS FOR THE USER TO COMPLETE. THE CODE SHOULD FULLY RUN \
        WITH NO ADDITIONAL MODIFICATIONS REQUIRED.
        """

        for i, example in enumerate(in_context_examples):
            input_text += f"Example {i+1}:\n{example['prompt_code']}\n\n Expected Output:\n{example['expected_code']}\n\n"

        input_text += f"Target Prompt:\n{target_prompt_code}\n"

        # Add the input-output pair to the dataset
        dataset.append({
            'input': input_text,
            'expected_output': expected_output_code,
            'actual_output': actual_output_code
        })
        break

    # Write the dataset to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['input', 'expected_output', 'actual_output']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for data in dataset:
            writer.writerow(data)

    print(f"Evaluation dataset created and saved to {output_csv}")

# Example usage
example_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'  # Folder with in-context examples
target_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_working_dataset'       # Folder with target prompts and expected outputs
output_csv = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset2.csv'             # Output CSV file path
actual_output_folder = r'C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\generated_inner_analyzers_ICL'

create_evaluation_dataset(example_folder, target_folder, actual_output_folder, output_csv, n=3)


Evaluation dataset created and saved to C:\Users\bhull\Desktop\UCLA Grad\Spring 2024\CS 219\219_final_project\LLM-assisted_mobile_trace_analysis\inner_dataset2.csv
